In [ ]:
# Multi-variable linear regression

In [5]:
import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

In [ ]:
# filename_queue = tf.train.string_input_producer(["file0.csv", "file1.csv"])
# #string_input_producer가 queue를 생성하고
 
# reader = tf.TextLineReader()
# key, value = reader.read(filename_queue)
 
# # Default values, in case of empty columns. Also specifies the type of the
# # decoded result.
# record_defaults = [[1], [1], [1], [1], [1]]
# col1, col2, col3, col4, col5 = tf.decode_csv(value, record_defaults=record_defaults)
# features = tf.stack([col1, col2, col3, col4])
 
# with tf.Session() as sess:
#   # Start populating the filename queue.
#   coord = tf.train.Coordinator()
#   threads = tf.train.start_queue_runners(coord=coord)
 
#   for i in range(1200):
#     # Retrieve a single instance:
#     example, label = sess.run([features, col5])
 
#   coord.request_stop()
#   coord.join(threads)

In [ ]:
# read의 각 실행은 파일로부터 한 줄을 읽습니다. 
# decode_csv는 결과를 tensor 목록으로 파싱합니다. 
# record_defaults 인자는 결과 tensor의 형식과 사용할 default value의 세트를 결정합니다. 
# record_defaults는 각 필드의 default값을 정해줄 뿐만 아니라 
# 데이터 타입을 지정해주는 역할도 합니다. 
# 위의 코드를 실행시키면 csv의 각 행을 파싱하여 
# 각 필드를 col1,2,3,4,5라는 필드에 리턴하게 됩니다.

# 반드시 queue로부터 읽기 위해서 run, eval을 호출하기 전에 
# tf.train.start_queue_runner를 큐를 채우기 위해서 호출해야 합니다. 
# 그렇지 않으면 read는 queue에 filename을 기다리는 동안 block 됩니다.

In [28]:
# 실제로 모델을 만들어서 학습을 할 때는 데이터의 양이 많기 때문에 
# 메모리에 모두 적재하고 학습을 할 수가 없고
# 파일에서 읽어들이면서 학습을 해야 한다
# 즉, 파일에서 데이터를 읽어가면서 순차적으로 모델에 feeding => 큐 사용

filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')
filename_queue

In [29]:
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)
# (<tf.Tensor 'ReaderReadV2:0' shape=() dtype=string>,
#  <tf.Tensor 'ReaderReadV2:1' shape=() dtype=string>)

In [30]:
record_defaults = [ [0.], [0.], [0.], [0.] ]
xy = tf.decode_csv(value, record_defaults=record_defaults)
# [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>,
#  <tf.Tensor 'DecodeCSV:1' shape=() dtype=float32>,
#  <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>,
#  <tf.Tensor 'DecodeCSV:3' shape=() dtype=float32>]

In [45]:
# xy[0:-1], xy[-1:]
# ([<tf.Tensor 'DecodeCSV_1:0' shape=() dtype=float32>,
#   <tf.Tensor 'DecodeCSV_1:1' shape=() dtype=float32>,
#   <tf.Tensor 'DecodeCSV_1:2' shape=() dtype=float32>],
#  [<tf.Tensor 'DecodeCSV_1:3' shape=() dtype=float32>])

train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

In [46]:
# placeholders for a tensor that will be always fed
X = tf.placeholder(tf.float32, shape = [None, 3])
Y = tf.placeholder(tf.float32, shape = [None, 1])

In [47]:
W = tf.Variable(tf.random_normal([3, 1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

In [48]:
# Hypothesis
hypothesis = tf.matmul(X, W) + b

In [49]:
# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [50]:
# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [52]:
# Launch the graph in a session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Start populating the filename queue
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess, coord = coord)

for step in range(2001) :
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    # print(x_batch)
    # print(y_batch)
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], 
                                   feed_dict = {X : x_batch, Y : y_batch})
#     if step % 10 == 0 :
#         print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

In [53]:
print(sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))
print(sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

[[189.5634]]
[[184.8686 ]
 [174.17464]]
